In [2]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.io import loadmat
from matplotlib import pyplot as plt

info = pd.read_csv("G:/Data/Supplementaries/zebel-paired.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'G:/Data/Supplementaries/zebel-paired.csv'

In [2]:
def movavg(inp, bl, ss):
    nbin = int(np.floor((inp.shape[2] - bl) / ss))
    out = np.nan * np.zeros((inp.shape[0], inp.shape[1], nbin))
    for ibin in range(nbin):
        out[:, :, ibin] = np.sum(inp[:, :, int(ibin*ss):int(ibin*ss)+bl], 2) / bl * 1000
    return out

# Split Half Data

In [28]:
def read_neuron(path):
    cm = loadmat(os.path.join(os.path.dirname(path), 'cm.mat'))['cm']
    nstim = np.unique(cm).max()
    
    nsample = 900
    data = np.nan * np.zeros([nstim, 1, nsample])
    
    ua = loadmat(path)['ua']
    cm = cm[:ua.shape[0]]
    
    split_1, split_2 = [], []
    for istim in np.arange(1, 166):
        X = ua[(cm==istim).flatten(), :]
        nrep = X.shape[0]
        
        first_half = X[:int(nrep/2), :]
        second_half = X[int(nrep/2):, :]

        split_1.append(first_half.mean(0))
        split_2.append(second_half.mean(0))
    
    if nrep < 3:
        print(path)
    return np.array(split_1), np.array(split_2)

In [37]:
monkey = "jenab"
select = "fast"

info = pd.read_csv(f'/Data/{select.capitalize()}/{monkey.capitalize()}/itcNeuralInfo.csv')

split1, split2 = [], []
for row in info.iterrows():
    spl1, spl2 = read_neuron(row[1][0])
    split1.append(spl1)
    split2.append(spl2)

split1 = np.transpose(split1, [1, 0, 2])
split2 = np.transpose(split2, [1, 0, 2])

C:\Users\User\AppData\Local\Temp/ipykernel_336/1769340678.py:19: RuntimeWarning: Mean of empty slice.
  split_1.append(first_half.mean(0))
C:\Users\User\anaconda3\envs\ipm\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


G:\Data\Fast\Jenab\2017-10-17_12-16\Trial\mu_it.mat


In [38]:
np.save('G:/Data/Fast/Jenab/itc-half-split-1.npy', split1)
np.save('G:/Data/Fast/Jenab/itc-half-split-2.npy', split2)

# Jenab - Fast

In [24]:
def read_neuron(path):    
    cm = loadmat(os.path.join(os.path.dirname(path), 'cm.mat'))['cm']
    nstim = np.unique(cm).max()
    
    nsample = 900
    data = np.nan * np.zeros([nstim, 1, nsample])
    
    ua = loadmat(path)['ua']
    cm = cm[:ua.shape[0]]
    for istim in np.arange(1, nstim + 1, 1):
        data[istim-1, 0, :] = ua[(cm == istim).squeeze(), :].mean(0)
        
    return movavg(data, 50, 1)

In [38]:
info = pd.read_csv('/Data/Fast/Jenab/itcNeuralInfo.csv')

itc = []
for row in info.iterrows():
    itc.append(read_neuron(row[1][0]))

itc = np.transpose(np.array(itc).squeeze(), [1, 0, 2])

In [39]:
info = pd.read_csv('/Data/Fast/Jenab/pfcNeuralInfo.csv')

pfc = []
for row in info.iterrows():
    pfc.append(read_neuron(row[1][0]))

pfc = np.transpose(np.array(pfc).squeeze(), [1, 0, 2])

In [42]:
np.save('G:/Data/Fast/Jenab/ITC.npy', itc)
np.save('G:/Data/Fast/Jenab/PFC.npy', pfc)

# Zebel - Fast

In [68]:
def read_neuron(path):    
    cm = loadmat(os.path.join(os.path.dirname(path), 'cm.mat'))['cm']
    nstim = np.unique(cm).max()
    
    nsample = 900
    data = np.nan * np.zeros([nstim, 1, nsample])
    
    ua = loadmat(path)['ua']
    cm = cm[:ua.shape[0]]
    for istim in np.arange(1, nstim + 1, 1):
        data[istim-1, 0, :] = ua[(cm == istim).squeeze(), :].mean(0)
        
    return movavg(data, 50, 1)

In [71]:
info = pd.read_csv('/Data/Fast/Zebel/itcNeuralInfo.csv')

itc = []
for row in info.iterrows():
    itc.append(read_neuron(row[1][0]))

itc = np.transpose(np.array(itc).squeeze(), [1, 0, 2])

In [70]:
info = pd.read_csv('/Data/Fast/Zebel/pfcNeuralInfo.csv')

pfc = []
for row in info.iterrows():
    pfc.append(read_neuron(row[1][0]))

pfc = np.transpose(np.array(pfc).squeeze(), [1, 0, 2])

In [72]:
np.save('G:/Data/Fast/Zebel/ITC.npy', itc)
np.save('G:/Data/Fast/Zebel/PFC.npy', pfc)

# Both - Fast

In [73]:
itc_z = np.load('G:/Data/Fast/Zebel/ITC.npy')
itc_j = np.load('G:/Data/Fast/Jenab/ITC.npy')
itc_z = itc_z[:itc_j.shape[0], :, :]

itc = np.concatenate([itc_z, itc_j], axis=1)

np.save('G:/Data/Fast/Both/ITC.npy', itc)

In [74]:
pfc_z = np.load('G:/Data/Fast/Zebel/PFC.npy')
pfc_j = np.load('G:/Data/Fast/Jenab/PFC.npy')
pfc_z = pfc_z[:pfc_j.shape[0], :, :]

pfc = np.concatenate([pfc_z, pfc_j], axis=1)

np.save('G:/Data/Fast/Both/PFC.npy', pfc)

# Jenab - Slow

In [45]:
def read_neuron(path):    
    cm = loadmat(os.path.join(os.path.dirname(path), 'cm.mat'))['cm']
    nstim = np.unique(cm).max()
    
    nsample = 1200
    data = np.nan * np.zeros([nstim, 1, nsample])
    
    ua = loadmat(path)['ua']
    cm = cm[:ua.shape[0]]
    for istim in np.arange(1, nstim + 1, 1):
        data[istim-1, 0, :] = ua[(cm == istim).squeeze(), :].mean(0)
        
    return movavg(data, 50, 1)

In [46]:
info = pd.read_csv('/Data/Slow/Jenab/itcNeuralInfo.csv')
itc = []
for row in info.iterrows():
    itc.append(read_neuron(row[1][0]))
itc = np.transpose(np.array(itc).squeeze(), [1, 0, 2])

info = pd.read_csv('/Data/Slow/Jenab/pfcNeuralInfo.csv')
pfc = []
for row in info.iterrows():
    pfc.append(read_neuron(row[1][0]))
pfc = np.transpose(np.array(pfc).squeeze(), [1, 0, 2])

In [47]:
np.save('G:/Data/Slow/Jenab/ITC.npy', itc)
np.save('G:/Data/Slow/Jenab/PFC.npy', pfc)

# Zebel - Slow

In [51]:
def read_neuron(path):    
    cm = loadmat(os.path.join(os.path.dirname(path), 'cm.mat'))['cm']
    nstim = np.unique(cm).max()
    
    nsample = 1200
    data = np.nan * np.zeros([nstim, 1, nsample])
    
    ua = loadmat(path)['ua']
    cm = cm[:ua.shape[0]]
    for istim in np.arange(1, nstim + 1, 1):
        data[istim-1, 0, :] = ua[(cm == istim).squeeze(), :].mean(0)
        
    return movavg(data, 50, 1)

In [52]:
info = pd.read_csv('/Data/Slow/Zebel/itcNeuralInfo.csv')
itc = []
for row in info.iterrows():
    itc.append(read_neuron(row[1][0]))
itc = np.transpose(np.array(itc).squeeze(), [1, 0, 2])

info = pd.read_csv('/Data/Slow/Zebel/pfcNeuralInfo.csv')
pfc = []
for row in info.iterrows():
    pfc.append(read_neuron(row[1][0]))
pfc = np.transpose(np.array(pfc).squeeze(), [1, 0, 2])

In [55]:
np.save('G:/Data/Slow/Zebel/ITC.npy', itc)
np.save('G:/Data/Slow/Zebel/PFC.npy', pfc)

# Both - Slow

In [64]:
itc_z = np.load('G:/Data/Slow/Zebel/ITC.npy')
itc_j = np.load('G:/Data/Slow/Jenab/ITC.npy')
itc_z = itc_z[:itc_j.shape[0], :, :]

itc = np.concatenate([itc_z, itc_j], axis=1)

np.save('G:/Data/Slow/Both/ITC.npy', itc)

In [67]:
pfc_z = np.load('G:/Data/Slow/Zebel/PFC.npy')
pfc_j = np.load('G:/Data/Slow/Jenab/PFC.npy')
pfc_z = pfc_z[:pfc_j.shape[0], :, :]

pfc = np.concatenate([pfc_z, pfc_j], axis=1)

np.save('G:/Data/Slow/Both/PFC.npy', pfc)

# Zebel - Pair

In [3]:
def read_session(path):
    fpath = os.path.join(path, 'Trial')
    
    cm = loadmat(os.path.join(fpath, 'cm.mat'))['cm']
    nstim = np.unique(cm).max()
    
    nsample = loadmat(os.path.join(fpath, 'mu_it.mat'))['ua'].shape[1]
    it = np.nan * np.zeros([nstim, len(list(glob.iglob(f'{fpath}/*_it*.mat'))), nsample])
    
    for i, u in enumerate(glob.iglob(f'{fpath}/*_it*.mat')):
        ua = loadmat(u)['ua']
        cm = cm[:ua.shape[0]]
        for istim in np.arange(1, nstim + 1, 1):
            it[istim-1, i, :] = ua[(cm == istim).squeeze(), :].mean(0)

    pfc = np.nan * np.zeros([nstim, len(list(glob.iglob(f'{fpath}/*pfc*.mat'))), nsample])
    for i, u in enumerate(glob.iglob(f'{fpath}/*pfc*.mat')):
        ua = loadmat(u)['ua']
        cm = cm[:ua.shape[0]]
        for istim in np.arange(1, nstim + 1, 1):
            pfc[istim-1, i, :] = ua[(cm == istim).squeeze(), :].mean(0)
        
    return cm, movavg(it, 50, 1), movavg(pfc, 50, 1)

In [4]:
monkey = "Zebel"
it, pfc = [], []
for r in info.iterrows():
    cm, it_, pfc_ = read_session(os.path.join(f"G:/Data/Slow/{monkey}", r[1][0]))
    it.append(it_)
    pfc.append(pfc_)

it = np.concatenate(it, axis=1)
pfc = np.concatenate(pfc, axis=1)

slow = {'it': it, 'pfc': pfc}

it, pfc = [], []
for r in info.iterrows():
    cm, it_, pfc_ = read_session(os.path.join(f"G:/Data/Fast/{monkey}", r[1][1]))
    it.append(it_)
    pfc.append(pfc_)

it = np.concatenate(it, axis=1)
pfc = np.concatenate(pfc, axis=1)

fast = {'it': it, 'pfc': pfc}

In [5]:
np.save('G:/Data/Paired/Zebel/slow-it.npy', slow['it'])
np.save('G:/Data/Paired/Zebel/fast-it.npy', fast['it'])
np.save('G:/Data/Paired/Zebel/slow-pfc.npy', slow['pfc'])
np.save('G:/Data/Paired/Zebel/fast-pfc.npy', fast['pfc'])